# Parsing Selenium

Написать программу, которая собирает входящие письма из своего или тестового почтового ящика и сложить данные о письмах в базу данных (от кого, дата отправки, тема письма, текст письма полный)
Логин тестового ящика: study.ai_172@mail.ru
Пароль тестового ящика: NextPassword172!!!

Написать программу, которая собирает «Новинки» с сайта техники mvideo и складывает данные в БД. Сайт можно выбрать и свой. Главный критерий выбора: динамически загружаемые товары

In [59]:
from selenium import webdriver
from selenium.common import exceptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from pymongo import MongoClient
from pprint import pprint

In [51]:
def parse_element(element, text):
    result = WebDriverWait(element, 500).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, text))).text
    return result

In [52]:
def _is_exists(name_tags, field):
    return bool(collection.find_one({name_tags: {"$in": [field]}}))

In [53]:
def parse_email(element):
    item = {}
    item['from_name'] = parse_element(
        driver, 'span[class="letter-contact"]')
    item['date'] = parse_element(
        driver,'div[class="letter__date"]')
    item['subject'] = parse_element(
        driver, 'h2[class="thread__subject"]')
    item['text_messege'] = parse_element(
        driver, 'div[class="letter-body__body-wrapper"]')
    return item

In [54]:
client = MongoClient('127.0.0.1', 27017)
DataBase = client['MailRu']
collection = DataBase['Letters']

In [55]:
options = webdriver.ChromeOptions()
options.add_argument("user-data-dir=C:\\Users\\Alexander\\AppData\\Local\\Google\\Chrome\\User Data\\Default")
driver = webdriver.Chrome(executable_path=r'.\chromedriver.exe', options=options)
driver.get("https://www.mail.ru")

In [56]:
# Insert Login
login_form = driver.find_element_by_css_selector(
        'input[class="email-input svelte-1eyrl7y"]'
)

login_form.send_keys('study.ai_172@mail.ru')

# Push button to password
try:
    mail_button = driver.find_element_by_css_selector(
        'button[class ="button svelte-1eyrl7y"]'
    )
except exceptions.NoSuchElementException:
    print('button not found')

mail_button.click()

# Wait button load
password_form = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.NAME,"password")))

password_form.send_keys('NextPassword172!!!')



# Push button to Enter
try:
    enter_button = driver.find_element_by_css_selector(
        'button[class="second-button svelte-1eyrl7y"]'
    )
except exceptions.NoSuchElementException:
    print('button not found')

enter_button.click()

inside_main_form = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR,'span[class="compose-button__txt"]')))

# Open first letter
try:
    FL_button = driver.find_element_by_css_selector(
        'a[class="llc js-tooltip-direction_letter-bottom js-letter-list-item llc_normal"]'
    )
except exceptions.NoSuchElementException:
    print('button not found')

FL_button.click()

In [57]:
for i in range(10):
    try:
        collection.insert_one(parse_email(driver))
        WebDriverWait(driver, 500).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'svg[class="ico ico_16-arrow-down ico_size_s"]'))).click()
    except exceptions.TimeoutException:
        print('E-mails are over')
        break

In [60]:
i = 0 
for item in collection.find({}):
    pprint(item)
    i += 1
    if i == 5:
        break

{'_id': ObjectId('612b51dcfea00145da271561'),
 'date': 'Вчера, 20:36',
 'from_name': 'Webdesigner News',
 'subject': '✏ How to Design a Character, React Compound Components, Buttons '
            'Generator, and more...',
 'text_messege': '  How to Design a Character\n'
                 'VECTORNATOR.IO    COMMENTS\n'
                 'Media Queries in Responsive Design: A Complete Guide (2021)\n'
                 'TORQUEMAG.IO    COMMENTS\n'
                 'Compound Components in React\n'
                 'SMASHINGMAGAZINE.COM    COMMENTS\n'
                 'Buttons Generator - 100+ Buttons You Can Use in your '
                 'Project\n'
                 'MARKODENIC.COM    COMMENTS\n'
                 'The Mind-Bendingly Difficult Material Design Quiz\n'
                 'WEBDESIGNERDEPOT.COM    COMMENTS\n'
                 '  See latest posts\n'
                 '    Home   About   FAQs   Legal   Privacy   RSS   Update '
                 'email frequency   Contact\n'
           